In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-70b-8192"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.


In [3]:
!python --version

Python 3.13.1


In [4]:
!pip install selenium webdriver-manager


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import sys
print(sys.executable)


C:\Python313\python.exe


In [35]:
!pip install selenium webdriver-manager


Defaulting to user installation because normal site-packages is not writeable


In [6]:
import selenium
print(selenium.__version__)


4.30.0


In [30]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://weworkremotely.com/remote-jobs/orca-international-freelancer-remote-graphic-designer")
page_data = loader.load().pop().page_content
print(page_data)


Remote Freelancer-Remote Graphic Designer at Orca InternationalJobs  Top Trending Remote Jobs New!All JobsProgramming Full-Stack ProgrammingFront-End ProgrammingBack-End ProgrammingDesignDevops and SysadminManagement and FinanceProductCustomer SupportSales and MarketingAll Other JobsFull-TimeContractCommunity BlogPodcastRemote Work TrendsEventsFind work Top 100 Remote CompaniesTop Remote Job SearchesAdvanced Job Search New! Automate your Job Search New!FAQ - Job SeekersRSS FeedRemote Work ResourcesCareer Services Resume Review Service New! Automate your Job Search New!Find talent Remote Hiring GuideHiring Remote Software DevelopersWhy Choose WWR?FAQ - Employers Save with Bundles  Sign in Post a job Sign in  Jobs Top Trending Remote JobsNew!All Jobs Programming Full-Stack ProgrammingFront-End ProgrammingBack-End ProgrammingDesignDevops and SysadminManagement and FinanceProductCustomer SupportSales and MarketingAll Other JobsFull-TimeContract Community BlogPodcastRemote Work TrendsEvents

In [31]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [71]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Freelancer-Remote Graphic Designer',
  'experience': '3+ years of experience in graphic design, preferably in digital marketing or ecommerce.',
  'skills': ['Proficiency in Adobe Creative Suite (Photoshop, Illustrator, InDesign, XD)',
   'Experience with Figma, Sketch, or other UI/UX design tools',
   'Knowledge of web design principles, typography, and colour theory',
   'Basic knowledge of HTML & CSS is a plus'],
  'description': 'As a Freelancer-Remote Graphic Designer, you will be responsible for designing, maintaining, and optimizing visual assets for websites, marketing materials, social media, and digital campaigns. You will collaborate closely with designers, developers, content creators, and marketing professionals to ensure a cohesive and high-quality brand identity for our clients.'},
 {'role': 'Senior UX/UI Designer',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Client Facing Business Analyst',
  'experience': None,
  'skills': None,

In [56]:
type(json_res)

list

In [72]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df



,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [73]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [80]:
if isinstance(job, list) and job:
    job_data = job[0]  # Extract the first job entry
    skills = job_data.get("skills", [])

    # Ensure skills is a list and join for querying
    query_text = [" ".join(skills)] if isinstance(skills, list) else []

    links = collection.query(query_texts=query_text, n_results=2).get("metadatas", [])
    print(links)
else:
    print("Error: job is empty or not a list.")




[[{'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/ios-ar-portfolio'}]]


In [82]:
job

[{'role': 'Freelancer-Remote Graphic Designer',
  'experience': '3+ years of experience in graphic design, preferably in digital marketing or ecommerce.',
  'skills': ['Proficiency in Adobe Creative Suite (Photoshop, Illustrator, InDesign, XD)',
   'Experience with Figma, Sketch, or other UI/UX design tools',
   'Knowledge of web design principles, typography, and colour theory',
   'Basic knowledge of HTML & CSS is a plus'],
  'description': 'As a Freelancer-Remote Graphic Designer, you will be responsible for designing, maintaining, and optimizing visual assets for websites, marketing materials, social media, and digital campaigns. You will collaborate closely with designers, developers, content creators, and marketing professionals to ensure a cohesive and high-quality brand identity for our clients.'},
 {'role': 'Senior UX/UI Designer',
  'experience': None,
  'skills': None,
  'description': None},
 {'role': 'Client Facing Business Analyst',
  'experience': None,
  'skills': None,

In [89]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Anuriti, a business development executive at Fosho. Fosho is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Fosho's portfolio: {link_list}
        Remember you are Anuriti, BDE at Fosho. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)



Here is the cold email:

Subject: Expert Graphic Design Services for Your Business

Dear [Client],

I came across your job posting for a Freelancer-Remote Graphic Designer and was impressed by your commitment to creating a cohesive brand identity. As a Business Development Executive at Fosho, I'd like to introduce you to our team of expert graphic designers who can help you achieve your design goals.

At Fosho, we have a proven track record of delivering high-quality design solutions that drive business results. Our team of experienced designers is well-versed in Adobe Creative Suite, Figma, Sketch, and other UI/UX design tools. We understand the importance of web design principles, typography, and color theory, and can create visually stunning assets that align with your brand identity.

Our team has a strong background in digital marketing and ecommerce, and we're confident that we can help you design, maintain, and optimize visual assets for your websites, marketing materials, socia